# Migration guide from qiskit-ibmq-provider to qiskit-ibm-provider


## Background

`qiskit-ibmq-provider` was the original implementation of the `Provider` interface in `qiskit-terra` and the main entry point for accessing IBM Quantum’s systems, simulators, and services. As IBM Quantum expanded, different disjoint services were added to `qiskit-ibmq-provider`, often causing confusion among users, and making new features difficult to find. 

With the introduction of [Qiskit Runtime](https://quantum-computing.ibm.com/lab/docs/iql/runtime/), we decided it was time to restructure the provider:

- The access to backend services (most of the submodules in the previous `qiskit.providers.ibmq`), is replaced by this [qiskit-ibm-provider](https://pypi.org/project/qiskit-ibm-provider/) package. It has a similar interface as the original `qiskit-ibmq-provider` (with `ibm-` instead of `ibmq-`) and gives direct access to the [IBM Quantum compute resources](https://quantum-computing.ibm.com/services/resources). If you need lower-level access to do experiments like device characterization or to research new error correction routines, `qiskit-ibm-provider` is more suitable for you. 
- The access to Qiskit Runtime service (formally in `qiskit.providers.ibmq.runtime`), goes into [qiskit-ibm-runtime](https://pypi.org/project/qiskit-ibm-runtime/). It provides access to managed services through the Qiskit Runtime primitives which are integrated with the latest quantum computing technologies, such as error  mitigation and correction. If you prefer getting high quality probability distribution or expectation values without having to optimize the circuits yourself, you may want to start here. 
- The access to the experiment service (formally in `qiskit.providers.ibmq.experiment`) goes into [qiskit-ibm-experiment](https://pypi.org/project/qiskit-ibm-experiment/).

This migration guide focuses on moving from `qiskit-ibmq-provider` to `qiskit-ibm-provider`. For the rest of the migration guides, check https://github.com/Qiskit/qiskit-ibmq-provider#migration%20guides 

## Quick Start

In [1]:
from qiskit import QuantumCircuit
from qiskit_ibm_provider import IBMProvider

# Save account credentials.
# IBMProvider.save_account(token=MY_API_TOKEN)

# Load previously saved account credentials.
provider = IBMProvider()

# Create a circuit
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
qc.measure_all()

# Select a backend.
backend = provider.get_backend("ibmq_qasm_simulator")
# Submit a job.
job = backend.run(qc)
# Get results.
print(job.result().get_counts())

{'11': 2047, '00': 1953}


## Summary of main changes

While the interfaces stay mostly the same, there are a few major changes that require your actions. 

- `qiskit-ibm-provider` is no longer part of the Qiskit meta package and will need to be installed separately.
- Importing and initializing a provider is different. 
- Some infrequently used methods are no longer supported.
- `qiskit-ibm-provider` uses the Qiskit Runtime API under the cover.

## Installation

You can install `qiskit-ibm-provider` using `pip`:

```
pip install qiskit-ibm-provider
```

## Initializing an account

When using `qiskit-ibmq-provider`, one usually manages accounts and gets a provider using the `IBMQ` global variable. In `qiskit-ibm-provider` the provider is created using the `IBMProvider` class.

**Previously:**

In [ ]:
from qiskit import IBMQ

# Enable an account without saving.
# IBMQ.enable_account(token=MY_API_TOKEN)

# Save account credentials.
# IBMQ.save_account(token=MY_API_TOKEN)

# Load a previously saved account.
provider = IBMQ.load_account()

# Select a different hub/group/project.
provider = IBMQ.get_provider(hub="ibm-q", group="open", project="main")

**New:**

In [17]:
from qiskit_ibm_provider import IBMProvider

# Enable an account without saving.
# IBMProvider.enable_account(token=MY_API_TOKEN)

# Save account credentials.
# IBMProvider.save_account(token=MY_API_TOKEN)

# Load a previously saved account.
provider = IBMProvider()

# Select a different hub/group/project.
provider = IBMProvider(instance="ibm-q/open/main")

## Loading account from environment variables

The environment variables used for account credentials have changed:

- `QE_TOKEN` is replaced by `QISKIT_IBM_TOKEN`
- `QE_HUB`, `QE_GROUP`, and `QE_PROJECT` is replaced by `QISKIT_IBM_INSTANCE`

## Basic usage

We demonstrate the basic flow for using the provider, which is essentially the same. We begin by creating a demonstration circuit generating a 2-qubit bell pair:

In [3]:
from qiskit import QuantumCircuit

qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
qc.measure_all()

The basic flow of getting a backend, running the circuit using it, getting a job, and obtaining the job results is the same.

**Previously:**

In [ ]:
from qiskit import IBMQ

provider = IBMQ.load_account()
backend = provider.get_backend("ibmq_qasm_simulator")
job = backend.run(qc)
print(job.result().get_counts())

**New:**

In [ ]:
from qiskit_ibm_provider import IBMProvider

provider = IBMProvider()
backend = provider.get_backend("ibmq_qasm_simulator")
job = backend.run(qc)
print(job.result().get_counts())

## Removed functionalities

- The following parameters are no longer supported in `backend.run()`:

    - `job_name`. You can use job tags instead.
    - `experiment_id`. You can use job tags instead.
    - `job_share_level`. This was deprecated in `qiskit-ibmq-provider` and removed in `qiskit-ibm-provider`.
    - `qobj`. This was deprecated in `qiskit-ibmq-provider` and removed in `qiskit-ibm-provider`.
    - `memory_slots`.
    - `memory_slot_size`.
    - `rep_time`. You can use `rep_delay` instead.
    - `live_data_enabled`.

- Since `job_name`, `job_share_level`, and `experiment_id` are no longer supported, their associated methods are removed, including:
    - using job name and experiment ID as filters in `backend.jobs()`
    - `job.update_name()`
    - `job.name()`
    - `job.share_level()`
    - `job.experiment_id`

- There is no longer a limit on the number of active jobs you can have on a backend, so the associated methods are removed, including:
    - `backend.job_limit()`
    - `backend.remaining_jobs_count()`
    - `backend.active_jobs()`

- Retrieving reservation information is no longer supported.

- Retrieving calibration data associated with a job, using `job.properties()`, is no longer supported. You can, however, use `backend.properties(datetime=JOB_EXECUTION_TS)` to retrieve this data.

- `job.scheduling_mode()` is no longer supported.

- `job.wait_for_final_state()` no longer supports a callback function.
